In [1]:
from selenium import webdriver
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

url = 'https://queapetito.com/ensaladas/'
ensaladas = webdriver.Firefox(executable_path='/home/dsc/Selenium/geckodriver')
ensaladas.get(url)

In [2]:
#realizo el webscrapping de la página de recetas, sección ensaladas

In [2]:
clases = ['ensaladas','carne', 'guisos-potajes', 'horno', 'celiacos','sopas-cremas', 'faciles-rapidas', 'legumbres','pescado','pasta-pizza', 'arroces-cereales', 'verduras',
         'huevos-lacteos','mariscos']
link = []
name_food = []
for clase in clases:
    url_base = 'https://queapetito.com/{0}'.format(clase)
    clase = webdriver.Firefox(executable_path='/home/dsc/Selenium/geckodriver')
    clase.get(url_base)
    
    # Web scrapper for infinite scrolling page 
    WebDriverWait(clase,10).until(EC.element_to_be_clickable((By.XPATH,"//div[@class='qc-cmp2-summary-buttons']/button['last child']"))).click()
    scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
    screen_height = clase.execute_script("return window.screen.height;")   # get the screen height of the web
    i = 1
    n = 0

    while True:
        # scroll one screen height each time
        scroll_height_0=clase.execute_script("return document.body.scrollHeight;")
        i += 1
        clase.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = clase.execute_script("return document.body.scrollHeight;")  
        # Break the loop when the total scroll height don't change for 10 times
        if scroll_height_0 == scroll_height:
            n += 1
            if n == 10:
                break
        else:
            n = 0
    clase_html = clase.page_source    
    food = BeautifulSoup(clase_html, 'html5lib')
    name_clase = food.find_all('a', {'class':'title'})
    clase_link = list(name_clase)
    for i in clase_link:
        for x in str(i).split('"'):
            if x[0:4]== 'http':
                link.append(x)
            if x[0]== '>':
                name_food.append(x[1:-4])
                


In [3]:
len(name_food), len(link)

(2711, 2711)

In [4]:
calories = []
for i in range(len(link)):
    r = requests.get(link[i])
    nutricional = BeautifulSoup(r.content, 'html5lib')
    calorie =nutricional.find_all('div', {'class':'comun-data nutrition'}) #informacion donde se encuentran las calorias    
    calorie_link = list(calorie)
    calorie_ch = []
    for y in str(calorie_link):
        if y.isnumeric():
            calorie_ch.append(y)
    calorie_complete = "".join(calorie_ch)
    calories.append(calorie_complete)

In [5]:
len(calories)

2711

In [6]:
df = pd.DataFrame(name_food, columns= ['name_food'] )

In [7]:
df.head()

,name_food
0,Ensalada de escarola y granada para navidad
1,Ensalada Templada de Pollo y Bacon
2,Ensalada de brócoli crudo
3,Ensalada de lechuga con manzana y pasas
4,Ensalada de col con queso


In [8]:
df['calories']= calories

In [9]:
df.head()

,name_food,calories
0,Ensalada de escarola y granada para navidad,210
1,Ensalada Templada de Pollo y Bacon,319
2,Ensalada de brócoli crudo,110
3,Ensalada de lechuga con manzana y pasas,198
4,Ensalada de col con queso,150


In [10]:
df['link']= link

In [11]:
df.head(10)

,name_food,calories,link
0,Ensalada de escarola y granada para navidad,210,https://queapetito.com/receta-ensalada-escarol...
1,Ensalada Templada de Pollo y Bacon,319,https://queapetito.com/receta-ensalada-templad...
2,Ensalada de brócoli crudo,110,https://queapetito.com/receta-ensalada-brocoli...
3,Ensalada de lechuga con manzana y pasas,198,https://queapetito.com/receta-ensalada-lechuga...
4,Ensalada de col con queso,150,https://queapetito.com/receta-ensalada-col-queso/
5,Ensalada de tomate y pimiento asado,205,https://queapetito.com/receta-ensalada-tomate-...
6,Ensalada de Trampó,58,https://queapetito.com/receta-ensalada-trampo/
7,Ensalada de lechuga con queso de cabra y jamón,190,https://queapetito.com/receta-ensalada-lechuga...
8,Ensalada de alubias blancas con pesto,520,https://queapetito.com/receta-ensalada-alubias...
9,Ensalada con queso de cabra y bacón,175,https://queapetito.com/receta-ensalada-queso-c...


In [15]:
df.to_csv('df_food_calories_link.csv')

In [18]:
df2=pd.read_csv('df_food_calories_link.csv', index_col=0)

In [42]:
len(df2)

2711

In [19]:
df2.head(10)

,name_food,calories,link
0,Ensalada de escarola y granada para navidad,210.0,https://queapetito.com/receta-ensalada-escarol...
1,Ensalada Templada de Pollo y Bacon,319.0,https://queapetito.com/receta-ensalada-templad...
2,Ensalada de brócoli crudo,110.0,https://queapetito.com/receta-ensalada-brocoli...
3,Ensalada de lechuga con manzana y pasas,198.0,https://queapetito.com/receta-ensalada-lechuga...
4,Ensalada de col con queso,150.0,https://queapetito.com/receta-ensalada-col-queso/
5,Ensalada de tomate y pimiento asado,205.0,https://queapetito.com/receta-ensalada-tomate-...
6,Ensalada de Trampó,58.0,https://queapetito.com/receta-ensalada-trampo/
7,Ensalada de lechuga con queso de cabra y jamón,190.0,https://queapetito.com/receta-ensalada-lechuga...
8,Ensalada de alubias blancas con pesto,520.0,https://queapetito.com/receta-ensalada-alubias...
9,Ensalada con queso de cabra y bacón,175.0,https://queapetito.com/receta-ensalada-queso-c...


In [41]:
len(df2['name_food'].unique())

2705

In [26]:
(df2.groupby(['name_food']).count())

,calories,link
name_food,,
4 Brochetas para Barbacoa,1,1
5 recetas con caviar,0,1
Abadejo a la cerveza,1,1
Abadejo a la gallega,1,1
Abadejo a la plancha con salsa,1,1
Abadejo al horno,1,1
Abadejo al horno con verduras,1,1
Abadejo en Salsa Verde,1,1
Abaja a la algecirena,0,1


In [47]:
df2.isna().sum()

name_food     0
calories     38
link          0
dtype: int64

In [50]:
df3= df2.dropna()

In [51]:
len(df3)

2673

In [62]:
print(len(df3), 'all dataset')
print(len(df3['name_food'].unique()), 'name_food unique')
print(len(df3['link'].unique()), 'link unique')

2673 all dataset
2667 name_food unique
2673 link unique


In [69]:
df3[df3.duplicated(subset=['name_food'])]

,name_food,calories,link
440,Chuletas de cordero a la plancha,280.0,https://queapetito.com/receta-chuletas-cordero...
580,Albóndigas en salsa de cebolla,140.0,https://queapetito.com/receta-albondigas-salsa...
827,Berenjenas rellenas de atún,310.0,https://queapetito.com/receta-berenjenas-relle...
1304,Alubias rojas estofadas,350.0,https://queapetito.com/receta-alubias-rojas-es...
1353,Estofado de besugo,146.0,https://queapetito.com/receta-estofado-besugo/
1632,Bacalao con nata,310.0,https://queapetito.com/receta-bacalao-nata/


In [70]:
df_vf= df3.drop_duplicates(subset=['name_food'])

In [71]:
print(len(df_vf), 'all dataset final version')
print(len(df_vf['name_food'].unique()), 'name_food unique')
print(len(df_vf['link'].unique()), 'link unique')

2667 all dataset final version
2667 name_food unique
2667 link unique


In [72]:
df_vf.to_csv('df_vf_food_calories_link.csv')

In [78]:
df_vf.dtypes

name_food     object
calories     float64
link          object
dtype: object

In [86]:
df_vf[df_vf['calories']<200]

,name_food,calories,link
2,Ensalada de brócoli crudo,110.0,https://queapetito.com/receta-ensalada-brocoli...
3,Ensalada de lechuga con manzana y pasas,198.0,https://queapetito.com/receta-ensalada-lechuga...
4,Ensalada de col con queso,150.0,https://queapetito.com/receta-ensalada-col-queso/
6,Ensalada de Trampó,58.0,https://queapetito.com/receta-ensalada-trampo/
7,Ensalada de lechuga con queso de cabra y jamón,190.0,https://queapetito.com/receta-ensalada-lechuga...
9,Ensalada con queso de cabra y bacón,175.0,https://queapetito.com/receta-ensalada-queso-c...
10,Ensalada de tomate y queso feta,198.0,https://queapetito.com/receta-ensalada-tomate-...
12,Ensalada de Lechuga Romana,67.0,https://queapetito.com/receta-ensalada-lechuga...
13,Ensalada de brócoli con atún y huevo duro,172.0,https://queapetito.com/receta-ensalada-brocoli...
14,Ensalada con arándanos,190.0,https://queapetito.com/receta-ensalada-arandanos/
